In [1]:
import torch
import jieba.posseg as pseg
import codecs
from gensim import corpora, models, similarities
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline

D:\program\Anaconda\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
df_record = pd.read_csv('admission_mr.csv')
df_record.head()

,ID,SYSTYPE,DATATYPE,PATIENT_ID,PATIENT_NO,INPATIENT_NO,INDEX2,PARENTNODENAME,NODENAME,NODEVALUE,...,OPERATOR_NAME,OPERATOR_TIME,NAME,UPDATED_ON,PROCESS_STATUS,SEQ_ID,HOSPITAL_CODE,OPERATOR_SYS_EMP_NO,DATA_VERSION,INP_PATIENT_NO
0,2199335,0,入院相关,0000545965,287627,ZY170000287627,曾*忠,入院病历\病史\既往史\手术外伤史,手术外伤史,NaN,...,何梦璋*,2012/12/27 星期四 12:56:41,再次或多次入院记录,05-6月 -18 02.53.06.000000 下午,NaN,31543,12440100455344205E,NaN,0,NaN
1,2199335,0,入院相关,0000545965,287627,ZY170000287627,曾*忠,入院病历\个人史\一般情况,一般情况,NaN,...,何梦璋*,2012/12/27 星期四 12:56:41,再次或多次入院记录,05-6月 -18 02.53.06.000000 下午,NaN,31544,12440100455344205E,NaN,0,NaN
2,2199339,0,入院相关,0000545965,287627,ZY170000287627,曾*忠,入院病历\体格检查一般情况\体位,体位,自主,...,何梦璋*,2012/12/27 星期四 12:59:44,再次或多次入院记录,05-6月 -18 02.53.06.000000 下午,NaN,31040,12440100455344205E,NaN,0,NaN
3,2199339,0,入院相关,0000545965,287627,ZY170000287627,曾*忠,入院病历\体格检查头面部\口唇,口唇,正常,...,何梦璋*,2012/12/27 星期四 12:59:44,再次或多次入院记录,05-6月 -18 02.53.06.000000 下午,NaN,31041,12440100455344205E,NaN,0,NaN
4,2199339,0,入院相关,0000545965,287627,ZY170000287627,曾*忠,入院病历\体格检查颈部\甲状腺,甲状腺,未及,...,何梦璋*,2012/12/27 星期四 12:59:44,再次或多次入院记录,05-6月 -18 02.53.06.000000 下午,NaN,31042,12440100455344205E,NaN,0,NaN


In [3]:
df_patient_disease_record = df_record[df_record['NODENAME'] == '现病史']
print(len(df_patient_disease_record))
df_patient_disease_record.head()

1630


,ID,SYSTYPE,DATATYPE,PATIENT_ID,PATIENT_NO,INPATIENT_NO,INDEX2,PARENTNODENAME,NODENAME,NODEVALUE,...,OPERATOR_NAME,OPERATOR_TIME,NAME,UPDATED_ON,PROCESS_STATUS,SEQ_ID,HOSPITAL_CODE,OPERATOR_SYS_EMP_NO,DATA_VERSION,INP_PATIENT_NO
275,2318913,0,入院相关,0080324728,365305,ZY010000365305,黄*强,入院病历\病史\现病史,现病史,患者1周前无明显诱因开始出现发热，最高体温37.9℃，无伴畏寒、寒战、皮疹，无明显时间规律，...,...,张鹤*,2013/2/28 星期四 16:54:40,入院记录,05-6月 -18 02.53.06.000000 下午,NaN,86784,12440100455344205E,NaN,0,NaN
315,2294384,0,入院相关,0000749211,321757,ZY060000321757,陈*成,入院病历\病史\现病史,现病史,患者2010-11因咳嗽咳痰就诊于当地医院，行胸部CT提示1、考虑双肺矽肺并双侧局部胸膜增厚...,...,叶枫**,2013/2/16 星期六 16:49:07,入院记录,05-6月 -18 02.53.06.000000 下午,NaN,89457,12440100455344205E,NaN,0,NaN
347,2329240,0,入院相关,0000033445,280478,ZY110000280478,吴*伙,入院病历\病史\现病史,现病史,患者7年前无明显诱因出现咳嗽、咳痰，为阵发性咳，咳声高亢，少量白泡沫痰，尚可咯出，伴气促，以...,...,朱琳**,2013/3/6 星期三 23:18:27,中医科入院记录,05-6月 -18 02.53.06.000000 下午,NaN,104804,12440100455344205E,NaN,0,NaN
353,2329166,0,入院相关,0000033445,280478,ZY110000280478,吴*伙,入院病历\病史\现病史,现病史,患者5年前无明显诱因出现咳嗽、咳痰，为阵发性咳，咳声高亢，少量白泡沫痰，尚可咯出，伴气促，以...,...,李振球,2013/3/5 星期二 16:38:59,再次或多次入院记录,05-6月 -18 02.53.06.000000 下午,NaN,104333,12440100455344205E,NaN,0,NaN
416,2333860,0,入院相关,0080326996,365630,ZY010000365630,张*焕,入院病历\病史\现病史,现病史,患者2月余前外伤致右胸疼痛伴全身多处疼痛，当时至梅州市人民医院就诊，胸部CT提示：“右侧血气...,...,李树本,2013/3/7 星期四 15:20:27,入院记录,05-6月 -18 02.53.06.000000 下午,NaN,103666,12440100455344205E,NaN,0,NaN


In [5]:
df_patient_disease_record_extracted = df_patient_disease_record[['INPATIENT_NO','NODEVALUE']]
print(len(df_patient_disease_record_extracted))
df_patient_disease_record_extracted.head()

1630


,INPATIENT_NO,NODEVALUE
275,ZY010000365305,患者1周前无明显诱因开始出现发热，最高体温37.9℃，无伴畏寒、寒战、皮疹，无明显时间规律，...
315,ZY060000321757,患者2010-11因咳嗽咳痰就诊于当地医院，行胸部CT提示1、考虑双肺矽肺并双侧局部胸膜增厚...
347,ZY110000280478,患者7年前无明显诱因出现咳嗽、咳痰，为阵发性咳，咳声高亢，少量白泡沫痰，尚可咯出，伴气促，以...
353,ZY110000280478,患者5年前无明显诱因出现咳嗽、咳痰，为阵发性咳，咳声高亢，少量白泡沫痰，尚可咯出，伴气促，以...
416,ZY010000365630,患者2月余前外伤致右胸疼痛伴全身多处疼痛，当时至梅州市人民医院就诊，胸部CT提示：“右侧血气...


In [6]:
df_patient_disease_record_extracted = df_patient_disease_record_extracted.drop_duplicates(['INPATIENT_NO'])
print(len(df_patient_disease_record_extracted))

1479


In [8]:
df_patient_disease_record_extracted = df_patient_disease_record_extracted.dropna()
print(len(df_patient_disease_record_extracted))

1421


In [12]:
words_record_list = []
for each in df_patient_disease_record_extracted['NODEVALUE']:
    temp_word_list = []
    words = pseg.cut(each)
    for word, flag in words:
        temp_word_list.append(word)
    words_record_list.append(temp_word_list)
    
print(words_record_list[:100])

[['患者', '1', '周前', '无', '明显', '诱因', '开始', '出现', '发热', '，', '最高', '体温', '37.9', '℃', '，', '无伴', '畏寒', '、', '寒战', '、', '皮疹', '，', '无', '明显', '时间', '规律', '，', '伴', '咳嗽', '，', '连声', '咳', '，', '伴咳', '黄色', '粘', '痰', '，', '无', '血丝', '，', '量', '一般', '，', '无', '明显', '时间', '规律', '，', '伴', '气促', '，', '无伴', '胸闷', '、', '心悸', '，', '无伴', '恶心', '、', '呕吐', '、', '腹痛', '、', '腹胀', '、', '腹泻', '，', '无伴', '尿频', '、', '尿急', '、', '尿痛', '，', '未予', '特殊', '处理', '，', '现', '为', '进一步', '诊治', '遂', '至', '我院', '就诊', '，', '查', '血常规', '：', '白细胞', '计数', '：', '16.49', '*', '10', '^', '9', '/', 'L', ',', '中性', '粒细胞', '比率', ':', '80.1', '%', '，', '胸片', '：', '两', '肺', '纹理', '增', '粗', '增多', '，', '拟', '两下', '肺', '散', '在', '少许', '炎症', '，', '门诊', '拟', '“', '双', '下', '肺炎', '”', '收治', '我', '科', '。', '患者', '自', '起病', '来', '，', '精神', '、', '睡眠', '尚可', '，', '胃纳', '一般', '，', '大小便', '如常', '，', '近期', '体重', '无', '明显', '变化', '。'], ['患者', '2010', '-', '11', '因', '咳嗽', '咳痰', '就诊', '于', '当地', '医院', '，', '行', '胸部', 'CT', '提示', '1', '、', '考虑', '双

In [13]:
df_patient_disease_record_extracted['desease_describe_words'] = np.nan
df_patient_disease_record_extracted.head()

,INPATIENT_NO,NODEVALUE,desease_describe_words
275,ZY010000365305,患者1周前无明显诱因开始出现发热，最高体温37.9℃，无伴畏寒、寒战、皮疹，无明显时间规律，...,NaN
315,ZY060000321757,患者2010-11因咳嗽咳痰就诊于当地医院，行胸部CT提示1、考虑双肺矽肺并双侧局部胸膜增厚...,NaN
347,ZY110000280478,患者7年前无明显诱因出现咳嗽、咳痰，为阵发性咳，咳声高亢，少量白泡沫痰，尚可咯出，伴气促，以...,NaN
416,ZY010000365630,患者2月余前外伤致右胸疼痛伴全身多处疼痛，当时至梅州市人民医院就诊，胸部CT提示：“右侧血气...,NaN
512,ZY010000365806,患者于5年前，出现咳嗽、咳痰，多于早上起床时，伴有流涕为清涕，无明显鼻塞，无流脓鼻涕。并且出...,NaN


In [15]:
df_patient_disease_record_extracted['desease_describe_words'] = words_record_list
df_patient_disease_record_extracted.head()

,INPATIENT_NO,NODEVALUE,desease_describe_words
275,ZY010000365305,患者1周前无明显诱因开始出现发热，最高体温37.9℃，无伴畏寒、寒战、皮疹，无明显时间规律，...,"[患者, 1, 周前, 无, 明显, 诱因, 开始, 出现, 发热, ，, 最高, 体温, ..."
315,ZY060000321757,患者2010-11因咳嗽咳痰就诊于当地医院，行胸部CT提示1、考虑双肺矽肺并双侧局部胸膜增厚...,"[患者, 2010, -, 11, 因, 咳嗽, 咳痰, 就诊, 于, 当地, 医院, ，,..."
347,ZY110000280478,患者7年前无明显诱因出现咳嗽、咳痰，为阵发性咳，咳声高亢，少量白泡沫痰，尚可咯出，伴气促，以...,"[患者, 7, 年前, 无, 明显, 诱因, 出现, 咳嗽, 、, 咳痰, ，, 为, 阵发..."
416,ZY010000365630,患者2月余前外伤致右胸疼痛伴全身多处疼痛，当时至梅州市人民医院就诊，胸部CT提示：“右侧血气...,"[患者, 2, 月余, 前, 外伤, 致, 右, 胸, 疼痛, 伴, 全身, 多处, 疼痛,..."
512,ZY010000365806,患者于5年前，出现咳嗽、咳痰，多于早上起床时，伴有流涕为清涕，无明显鼻塞，无流脓鼻涕。并且出...,"[患者, 于, 5, 年前, ，, 出现, 咳嗽, 、, 咳痰, ，, 多于, 早上, 起床..."


In [20]:
word_bag_set = set()
for each in words_record_list:
    for word in each:
        word_bag_set.add(word)

print(len(word_bag_set))
print(word_bag_set)

6764
{'上呼吸道', '行走', '第', '特别', '隐痛', '现上', '后查', '食量', '更为', '血小板', '坏死', '未予', '迭及', '同种', '脓', '最', '结肠炎', '聚集', '畅后', 'CRP141', '子宫', '早晨', '爬楼', '３', '致敏性', '5', '6.3', '降糖药', '刺', '化生', '短暂', '下', 'P', '粘量', '淋巴瘤', '剂量', '其余', '６', '病患者', '质粘量', '无端', '气量', '珠江', '触及', '双介', '待定', '肠溶', '解放军', '发热时', 'ICU', '最初', '扶正', '现症', '选择性', '即易', '大部分', '继之', '余分钟', '发红', '558', '布地', '参芪', '理疗', '分', '抖动', '交流', '腹泻', 'mgqd', '肠镜', '喷雾剂', '腋', '近几年来', '县', '频', '利胆', '17.8', '前天', '呼之', '那曲', 'TP53', '停下', '分叶', '#', '米乐松', '上腹', '发于', '转变', '纤支镜', 'CYFRA', '施', '叁', '念珠菌', '陈', '脂肪瘤', '多难', '香烟', '等', '冬天', '变细', '红豆杉', '15.02', '赤脚医生', '黄素', '寻求', '中医医院', '心率', '可吸入', '鹤山市', '牙龈', '2016.03.08', '浮肿', '证', '较易', '升结肠', '诊', '右上', '7.337', '回声', '游离', '睡眠', '巨大', '常', '淋巴细胞', '不祥', '组织细胞', '剑', '沙胺', '序', '美洛', '扶行', '92.9', '查双', '正位', '对称性', '影响', '导联', '需在', '室行', '毛细血管', '叙说', '多巴胺', '机械', '搏动', '支架', '参加', '太罗后', '滑脱', '服酮', '-', '早上', '学', '美唑', 'OSAHS', '饮水', '先锋', '.8', '麻仁', '隙性'

# 2）word embedding

In [25]:
count_of_word_bag = len(word_bag_set)
word_to_ix = { word: i for i, word in enumerate(word_bag_set) }
ix_to_word = { i: word for i, word in enumerate(word_bag_set) }
print(ix_to_word)

{0: '上呼吸道', 1: '行走', 2: '第', 3: '特别', 4: '隐痛', 5: '现上', 6: '后查', 7: '食量', 8: '更为', 9: '血小板', 10: '坏死', 11: '未予', 12: '迭及', 13: '同种', 14: '脓', 15: '最', 16: '结肠炎', 17: '聚集', 18: '畅后', 19: 'CRP141', 20: '子宫', 21: '早晨', 22: '爬楼', 23: '３', 24: '致敏性', 25: '5', 26: '6.3', 27: '降糖药', 28: '刺', 29: '化生', 30: '短暂', 31: '下', 32: 'P', 33: '粘量', 34: '淋巴瘤', 35: '剂量', 36: '其余', 37: '６', 38: '病患者', 39: '质粘量', 40: '无端', 41: '气量', 42: '珠江', 43: '触及', 44: '双介', 45: '待定', 46: '肠溶', 47: '解放军', 48: '发热时', 49: 'ICU', 50: '最初', 51: '扶正', 52: '现症', 53: '选择性', 54: '即易', 55: '大部分', 56: '继之', 57: '余分钟', 58: '发红', 59: '558', 60: '布地', 61: '参芪', 62: '理疗', 63: '分', 64: '抖动', 65: '交流', 66: '腹泻', 67: 'mgqd', 68: '肠镜', 69: '喷雾剂', 70: '腋', 71: '近几年来', 72: '县', 73: '频', 74: '利胆', 75: '17.8', 76: '前天', 77: '呼之', 78: '那曲', 79: 'TP53', 80: '停下', 81: '分叶', 82: '#', 83: '米乐松', 84: '上腹', 85: '发于', 86: '转变', 87: '纤支镜', 88: 'CYFRA', 89: '施', 90: '叁', 91: '念珠菌', 92: '陈', 93: '脂肪瘤', 94: '多难', 95: '香烟', 96: '等', 97: '冬天', 98: '变细', 

In [27]:
data = []

for  ii in range(len(words_record_list)):
    words = words_record_list[ii]
    for i in range(2, len(words) - 2):
        target = [ words[i-2], words[i-1], words[i+1], words[i+2] ]
        context = words[i]
        data.append( (context, target) )
        
print(data[:5])

[('周前', ['患者', '1', '无', '明显']), ('无', ['1', '周前', '明显', '诱因']), ('明显', ['周前', '无', '诱因', '开始']), ('诱因', ['无', '明显', '开始', '出现']), ('开始', ['明显', '诱因', '出现', '发热'])]


In [28]:
# 为了训练把数据改成如下形式
data_t = []
for  ii in range(len(words_record_list)):
    words = words_record_list[ii]
    for i in range(2, len(words) - 2):
        target = [ words[i-2], words[i-1], words[i+1], words[i+2] ]
        context = words[i]
        for iii in range(len(target)):
            data_t.append( (context, target[iii]))

        
print(data_t[:20])

# data
# [('生活', ['我', '知道', '是', '十分']), ('是', ['知道', '生活', '十分', '艰难']), ('十分', ['生活', '是', '艰难', '的']), ('艰难', ['是', '十分', '的', '不论']), ('的', ['十分', '艰难', '不论', '我'])]
# data_t
# [('生活', '我'), ('生活', '知道'), ('生活', '是'), ('生活', '十分'), ('是', '知道'), ('是', '生活'), ('是', '十分'), ('是', '艰难'), ('十分', '生活'), ('十分', '是'), ('十分', '艰难'), ('十分', '的'), ('艰难', '是'), ('艰难', '十分'), ('艰难', '的'), ('艰难', '不论'), ('的', '十分'), ('的', '艰难'), ('的', '不论'), ('的', '我')]



[('周前', '患者'), ('周前', '1'), ('周前', '无'), ('周前', '明显'), ('无', '1'), ('无', '周前'), ('无', '明显'), ('无', '诱因'), ('明显', '周前'), ('明显', '无'), ('明显', '诱因'), ('明显', '开始'), ('诱因', '无'), ('诱因', '明显'), ('诱因', '开始'), ('诱因', '出现'), ('开始', '明显'), ('开始', '诱因'), ('开始', '出现'), ('开始', '发热')]


In [ ]:
# Word2Vec Model.
class Word2Vec(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(Word2Vec, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)
        
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = self.linear(embeds)
        log_probs = F.log_softmax(out)
        return log_probs

In [ ]:
# Bi-LSTM 模型 
# 实现起来很简单 只需把 nn.RNN 替换成 nn.LSTM 以及
# 添加属性 bidirectional= True
# 倍化 hidden_size
class RNN(nn.Module):
  
    def __init__(self, batch_size, num_tokens, embed_size, hidden):        
        super(RNN, self).__init__()
        
        self.batch_size = batch_size
        self.num_tokens = num_tokens
        self.embed_size = embed_size
        self.hidden = hidden
        
        self.lookup = nn.Embedding(num_tokens, embed_size)
        self.bi_LSTM = nn.LSTM(embed_size, word_gru_hidden, bidirectional= True)
        
    def forward(self, embed, state_word):
        embedded = self.lookup(embed)
        output_word, state_word = self.bi_LSTM(embedded, state_word)       
        return output_word, state_word
    
    def init_hidden(self):
        return Variable(torch.zeros(2, self.batch_size, self.hidden))